In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install git+https://github.com/qubvel/segmentation_models

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/qubvel/segmentation_models to /tmp/pip-req-build-mf_w83zt
  Running command git clone --filter=blob:none --quiet https://github.com/qubvel/segmentation_models /tmp/pip-req-build-mf_w83zt
  Resolved https://github.com/qubvel/segmentation_models to commit e951c6747f75fa9e7240816d1c79dd2e66813123
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done


In [9]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
# from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Add, Multiply, Reshape, Lambda
from tensorflow.keras.models import Model
from efficientnet.tfkeras import EfficientNetB3
from tensorflow.keras.optimizers import AdamW

In [10]:
# Load the data from the numpy files
x_train = np.load('/content/drive/MyDrive/np_images/x_train.npy')
y_train = np.load('/content/drive/MyDrive/np_images/y_train.npy')
x_test  = np.load('/content/drive/MyDrive/np_images/x_test.npy')
y_test  = np.load('/content/drive/MyDrive/np_images/y_test.npy')

In [11]:
# resize
x_train = tf.image.resize(x_train, size=(224, 224)) / 255.0
x_test = tf.image.resize(x_test, size=(224, 224)) / 255.0

In [12]:
def preprocess_image(img):
    # Crop the center 80% of the image
    #print(img.shape)
    height, width, channels = img.shape
    crop_height = int(height * 0.8)
    crop_width = int(width * 0.8)
    start_height = int((height - crop_height) / 2)
    start_width = int((width - crop_width) / 2)
    img = img[start_height:start_height+crop_height, start_width:start_width+crop_width, :]
    
    # Apply Gaussian blur with kernel size 3
    img = cv2.GaussianBlur(img, (3, 3), 0)

    img = cv2.resize(img, (224, 224))
    
    # Rescale the pixel values to [0, 1]
    img = img / 255.
    
    # Rescale the pixel values to [0, 1]
    
    return img

In [13]:
from keras.preprocessing.image import ImageDataGenerator
def create_datagen():
    return ImageDataGenerator(
        horizontal_flip=True,   # randomly flip images
        vertical_flip=True,     # randomly flip images

        preprocessing_function = preprocess_image
    )
# Using original generator
data_generator = create_datagen().flow(x_train, y_train,batch_size=32)
print("Image data augmentated ...")

Image data augmentated ...


In [17]:
def efficientnet(num_classes):
    base_model = EfficientNetB3(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    output = Dense(num_classes, activation='softmax')(x)

    model = keras.models.Model(inputs=base_model.input, outputs=output)

    for layer in base_model.layers:
        layer.trainable = False

    return model


# Split data into train and validation sets
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(32)

test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.batch(32)

# f1 score
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior() 
import keras.backend as K

def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

# ---------------------------------------------------------------------
# Create model
model = efficientnet(num_classes=5)
model.compile(loss='categorical_crossentropy', optimizer=AdamW(lr=0.0001), metrics=['accuracy',f1])

# Train model
history = model.fit(train_dataset, epochs=100, validation_data=test_dataset)

# Evaluate model on test set
# test_loss, test_acc = model.evaluate(test_dataset)
# print('Test loss:', test_loss)
# print('Test accuracy:', test_acc)


Epoch 1/100
92/92 [==============================] - 29s 201ms/step - loss: 0.8603 - accuracy: 0.6787 - f1: 0.2801 - val_loss: 0.7409 - val_accuracy: 0.7149 - val_f1: 0.3054
Epoch 2/100
92/92 [==============================] - 13s 140ms/step - loss: 0.6941 - accuracy: 0.7368 - f1: 0.3594 - val_loss: 0.7144 - val_accuracy: 0.7381 - val_f1: 0.4136
Epoch 3/100
92/92 [==============================] - 13s 143ms/step - loss: 0.6718 - accuracy: 0.7392 - f1: 0.3799 - val_loss: 0.6769 - val_accuracy: 0.7531 - val_f1: 0.3748
Epoch 4/100
92/92 [==============================] - 13s 141ms/step - loss: 0.6407 - accuracy: 0.7586 - f1: 0.3981 - val_loss: 0.6646 - val_accuracy: 0.7667 - val_f1: 0.4519
Epoch 5/100
92/92 [==============================] - 13s 138ms/step - loss: 0.6242 - accuracy: 0.7593 - f1: 0.4263 - val_loss: 0.7033 - val_accuracy: 0.7653 - val_f1: 0.4029
Epoch 6/100
92/92 [==============================] - 13s 139ms/step - loss: 0.5993 - accuracy: 0.7685 - f1: 0.4403 - val_loss: 0.6

In [18]:
# convert the history.history dict to a pandas DataFrame:  
import pandas as pd   
hist_df = pd.DataFrame(history.history) 

#  saving history to csv file
hist_csv_file = '/content/drive/MyDrive/efficientnet/efficientnet_history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

# saving the model
model.save('/content/drive/MyDrive/efficientnet/effiecientnet.h5')
